In [38]:
%%time
%pdb on

import pandas as pd
import numpy as np
import inro.modeller as _m
import os

# Initialize the Modeller
modeller = _m.Modeller()
import_network_tool = modeller.tool("tmg.input_output.import_network_package")
export_network_tool = modeller.tool('tmg.input_output.export_network_package')


print "starting...."
print 

parameters = {
    "IMPORT_NETWORK_FLAG":True,
    "EXPORT_NETWORK_FLAG": True,
    "mode_to_modify": "r",  
    "network_folder": "sse_networks",  
    "export_network_folder": "exported_networks",  
    "station_to_node_file": "inputs/station_to_stop_updated.csv", 
    # "segment_file": "inputs/segment.csv",
    # "header_file": "inputs/header.csv", 
    "capacity_file": "inputs/vehicle_fix_final.csv", 
    "source_scenarios": [
        {"time_period": "AM", "scenario_id": 10},
        {"time_period": "Midday", "scenario_id": 20},
        {"time_period": "PM", "scenario_id": 30},
        {"time_period": "Evening", "scenario_id": 40},
        {"time_period": "Overnight", "scenario_id": 50},
    ],

}

def import_network(network_folder, network_file, scenario_id):
    """Imports a network package into a specified scenario."""
    network_package_path = os.path.join(network_folder, network_file)
    scenario_name = network_file[:-4]
    import_network_tool(network_package_path, scenario_id, 'OVERWRITE', True, scenario_name)
    print "Imported network for scenario {}".format(scenario_id)

def load_scenario(scenario_id):
    scenario = modeller.emmebank.scenario(scenario_id)
    if not scenario:
        raise Exception("Scenario {} was not found!".format(scenario_id))
    return scenario

def change_vehicle_properties(vehicle, description="", auto_eq=0.0, seated_cap=0, total_cap=0):
    
    if description:
        vehicle.description = description
    
    if auto_eq != 0.0:
        vehicle.auto_equivalent = auto_eq
        
    if seated_cap != 0:
        vehicle.seated_capacity = seated_cap
    
    # Ensure total capacity is not less than seated capacity
    total_cap = max(total_cap, seated_cap)
    
    if total_cap != 0:
        vehicle.total_capacity = total_cap   

def change_vehicle_properties(vehicle, description="", auto_eq=0.0, seated_cap=0, total_cap=0):
        
        if description:
            vehicle.description = description
        
        if auto_eq != 0.0:
            vehicle.auto_equivalent = auto_eq
            
        if seated_cap != 0:
            vehicle.seated_capacity = seated_cap
        
        # Ensure total capacity is not less than seated capacity
        total_cap = max(total_cap, seated_cap)
        
        if total_cap != 0:
            vehicle.total_capacity = total_cap

def export_network(scenario_number, folder, time_peirod, attributes_to_export):
    file_name = "Modified {} Network.nwp".format(time_peirod)
    file_path = os.path.join(folder, file_name)

    try:
        # Re-load the tool each time to avoid stateful errors
        export_network_tool(scenario_number, file_path, attributes_to_export)
        print "Exported {}".format(file_name)
    except:
        print "Error encountered exporting %s network" % file_name

#load files
capacity_df = pd.read_csv(parameters["capacity_file"])

capacity_df.set_index('name', inplace=True)
capacity_dict = {consist_type: row.to_dict() for consist_type, row in capacity_df.iterrows()}

# Import Networks if Flag is True
if parameters["IMPORT_NETWORK_FLAG"]:
    for network_file in os.listdir(parameters["network_folder"]):
        for scenario in parameters["source_scenarios"]:
            if scenario["time_period"] in network_file:
                print("Found source network: {} for scenario {}".format(network_file, scenario["scenario_id"]))
                import_network(parameters["network_folder"], network_file, scenario["scenario_id"])
                
# Load Scenarios and Process Network
source_scenario_list = []
for source_scenarios in parameters["source_scenarios"]:
    source_scenario = load_scenario(source_scenarios["scenario_id"])
    source_scenario_list.append(source_scenario)

for i, source_scenario in enumerate(source_scenario_list):
    source_network = source_scenario.get_network()
        
    new_vehicle_id_dict = {}
    existing_veh_desc = [str(veh.description.lower()) for veh in source_network.transit_vehicles()]
    existing_veh_id = sorted([veh.number for veh in source_network.transit_vehicles()])
    temp_vehs = []
    new_veh_nums = []
    max_existing_veh = 100

    for vehicle_desc, veh_attrib in capacity_dict.iteritems():
        try:
            if veh_attrib["id"] != 999:
#                 new_veh_id = max(existing_veh_id) + 10

                if max(existing_veh_id) < 100:
                    new_veh_id = 101
                else: 
                    new_veh_id = max(existing_veh_id) + 1
                    
                new_vehicle = source_network.create_transit_vehicle(new_veh_id, veh_attrib["mode"])
                seated_cap = veh_attrib["seat"]
                total_cap = veh_attrib["capacity"]
                description = vehicle_desc.strip()
                change_vehicle_properties(new_vehicle,
                                       description=description,
                                       auto_eq=0.0,
                                       seated_cap=seated_cap,
                                       total_cap=total_cap)
                existing_veh_id.append(new_veh_id)
                if new_vehicle not in temp_vehs and int(veh_attrib["id"]) not in new_veh_nums:
                    temp_vehs.append(new_vehicle)
                    new_veh_nums.append(int(veh_attrib["id"]))

                dependants = [line for line in source_network.transit_lines() if line.vehicle.number == veh_attrib["old_id"]]
#                 if veh_attrib["old_id"] ==15:
#                     print "dependants", dependants
                for line in dependants:
                    line.vehicle = new_vehicle
                source_network.delete_transit_vehicle(veh_attrib["old_id"])
                changed.append(veh_attrib["old_id"])
        except Exception as e:
            print
#             print "A: There was an error with old veh \'{}\' with veh name \'{}\'. Here is the error  \'{}\'".format( veh_attrib["old_id"],
#                                                                                        veh_attrib["old_name"], e)
    source_scenario.publish_network(source_network)
    source_network = source_scenario.get_network()
    
    issue_list = [50, 49,21,14,13]
    swap_list = [41,82,71,71,72]
    for vehicle_desc, veh_attrib in capacity_dict.iteritems():
        
        try: 
            if veh_attrib["old_id"] == 2:
                source_network.delete_transit_vehicle(veh_attrib["old_id"])
                
            if veh_attrib["id"] == 999 and veh_attrib["old_id"] in issue_list:
                veh_num = swap_list[issue_list.index(veh_attrib["old_id"])]
                
                for j, new_veh_num in enumerate(new_veh_nums):
                    if new_veh_num == veh_num:
                        new_vehicle =  temp_vehs[j]
                        break
                dependants = [line for line in source_network.transit_lines() if line.vehicle.number == veh_attrib["old_id"]]
                for line in dependants:
                    line.vehicle = new_vehicle
                source_network.delete_transit_vehicle(veh_attrib["old_id"])
        except Exception as e:
            print
            
#             print "B: There was an error with old veh \'{}\' with veh name \'{}\'. Here is the error  \'{}\'".format( veh_attrib["old_id"],
#                                                                                        veh_attrib["old_name"], e)
    print "CHECK NOW!!!"
    source_scenario.publish_network(source_network)
    
    source_network = source_scenario.get_network()
    
    track = []
    for temp_veh, new_veh_id in zip(temp_vehs, new_veh_nums):
        try:
            if new_veh_id not in track:
                new_vehicle_replaced = source_network.create_transit_vehicle(new_veh_id, temp_veh.mode.id)
                track.append(new_veh_id)
                
                for att in new_vehicle_replaced.network.attributes('TRANSIT_VEHICLE'):
                    new_vehicle_replaced[att] = temp_veh[att]
                
                dependants = [line for line in source_network.transit_lines() if line.vehicle.number == temp_veh.number]
                for line in dependants:
                    line.vehicle = new_vehicle_replaced
                print "deleting vehicle: ", temp_veh.number
                source_network.delete_transit_vehicle(temp_veh.number)
                
        except Exception as e:
            print "error from here:", e 
        
    source_scenario.publish_network(source_network)   
    if parameters["EXPORT_NETWORK_FLAG"]:
        time_period = parameters["source_scenarios"][i]["time_period"]
        export_network(source_scenario.id, parameters["export_network_folder"], time_period, "ALL")

Automatic pdb calling has been turned ON
starting....

Found source network: Assigned AM Peak Network.nwp for scenario 10
Imported network for scenario 10













































CHECK NOW!!!
deleting vehicle:  101
deleting vehicle:  102
deleting vehicle:  103
deleting vehicle:  104
deleting vehicle:  105
deleting vehicle:  106
deleting vehicle:  107
deleting vehicle:  108
deleting vehicle:  109
deleting vehicle:  110
deleting vehicle:  111
deleting vehicle:  112
deleting vehicle:  113
deleting vehicle:  114
deleting vehicle:  115
deleting vehicle:  116
deleting vehicle:  117
deleting vehicle:  118
deleting vehicle:  119
deleting vehicle:  120
deleting vehicle:  121
deleting vehicle:  122
deleting vehicle:  123
deleting vehicle:  124
deleting vehicle:  125
deleting vehicle:  126
deleting vehicle:  127
deleting vehicle:  128
deleting vehicle:  129
deleting vehicle:  130
deleting vehicle:  131
deleting vehicle:  132
deleting vehicle:  133
deleting vehicle:  134
delet